YOU CAN'T PROPERLY RUN THIS CODE HERE IN KAGGLE, USE YOUR LOCAL MACHINE (don't require gpu)

For this simple project we need cv2 for reading the image we want to process, displaying the image, setting a callback when the mouse is clicked , getting the RGB values for the pixel clicked and displaying the name of the color and its RGB values.
We also want to use panda for fast reading the csv file that contain 865 color names along with their RGB values

In [1]:
import cv2
import pandas as pd

reading the image(specify your image path)

In [2]:
img_path = "colorpic.jpg"
img = cv2.imread(img_path)

Reading csv file with pandas and giving names to each column

In [3]:
index=["color","color_name","hex","R","G","B"]
csv = pd.read_csv('colors.csv', names=index, header=None)

In the RGB space color colors are coded on 24 bits, that gives us 16,581,375 color. we can't name every single one, in our csv file we only have 865 color name. Thus Some RGB values will share the same name. What we need to do now is to assign every RGB value a name. We do that by calculating the distance from all colors and get the nearest color.

In [4]:
def getColorName(R,G,B):
    minimum = 1000
    for i in range(len(csv)):
        d = abs(R- int(csv.loc[i,"R"])) + abs(G- int(csv.loc[i,"G"]))+ abs(B- int(csv.loc[i,"B"]))
        if(d<minimum):
            minimum = d
            color_name = csv.loc[i,"color_name"]
    return color_name

some variables that help us know if the user clicked on the image (clicked) and if so (clicked == True) where (xpos,ypos) and what are RGB values (r,g,b) of that pixel

In [5]:
clicked = False
r = g = b = xpos = ypos = 0

a callbalck function that monitor events of the mouse. We only care for mouse double lift click. If this event occur we update our variables.

In [6]:
def draw_function(event, x,y,flags,param):
    if event == cv2.EVENT_LBUTTONDBLCLK:
        global b,g,r,xpos,ypos, clicked
        clicked = True
        xpos = x
        ypos = y
        b,g,r = img[y,x]
        b = int(b)
        g = int(g)
        r = int(r)

Creating a window in for the image setting a callback function for mouse event.

In [7]:
cv2.namedWindow('color detection')
cv2.setMouseCallback('color detection',draw_function)

Using the cv2.imshow() function, we draw the image on the window. When the user double clicks the window (clicked==True), we draw a rectangle (cv.rectangle()) and update the color_name (using getColorName function) and RGB values on the window

In [8]:
while(1):
    
    cv2.imshow("color detection",img)
    if (clicked):
        #cv2.rectangle(image, startpoint, endpoint, color, thickness)-1 fills entire rectangle 
        cv2.rectangle(img,(20,20), (750,60), (b,g,r), -1)
        color_name = getColorName(r,g,b) + ' R='+ str(r) +  ' G='+ str(g) +  ' B='+ str(b)
        #cv2.putText(img,text,start,font(0-7),fontScale,color,thickness,lineType )
        cv2.putText(img, color_name,(50,50),2,0.8,(255,255,255),2,cv2.LINE_AA)
        #For very light colours we will display text in black colour
        if(r+g+b>=600):
            cv2.putText(img, color_name,(50,50),2,0.8,(0,0,0),2,cv2.LINE_AA)
            
        clicked=False
    #when user hit esc
    if cv2.waitKey(20) & 0xFF ==27:
        break
    
cv2.destroyAllWindows()

KeyboardInterrupt: 

After inspecting the colors in the image, lets say we want to know all places where a a color show

we transform the image and that specific color to hsv space color.we set [H-10, 100,100] and [H+10, 255, 255] as lower bound and upper bound respectively so that only colors in that range stay after wwe apply cv2.inRange()

In [9]:
import numpy as np
hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
color = np.uint8([[[b,g,r]]])
hsv_color = cv2.cvtColor(color,cv2.COLOR_BGR2HSV)
lower_range = np.array([hsv_color[0][0][0] - 10,100,100])
upper_range = np.array([hsv_color[0][0][0] + 10,255,255])

mask = cv2.inRange(hsv, lower_range, upper_range)

cv2.imshow('mask', mask)
cv2.waitKey(0)
cv2.destroyAllWindows()

KeyboardInterrupt: 